In [1]:
import pandas as pd 
import numpy as np
import PyPDF2
import os
import spacy
import re
import string
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

# own module
import modules.pipeline as pipeline
import modules.preprocessing as preprocess

ModuleNotFoundError: No module named 'modules'

In [4]:
preprocess.extract_content()

<module 'modules.preprocessing' from 'C:\\Users\\Pieter-Jan\\Documents\\Carreer\\Candriam\\nlp\\ESG\\topic_sentiment_analysis\\modules\\preprocessing.py'>

In [16]:
%reload_ext autoreload
%autoreload 2

In [22]:
nlp = preprocess.load_spacy_model(model="en_core_web_sm")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [189]:
paragraphs[0]

' 2019 Sustainability Report  TABLE OF CONTENTS 3Our Sustainability Journey 3 Milestones 4 Our Business 5 From Our CEO 6Approach to Sustainability 6 ESG Integration 8 2019 Sustainability Highlights'

In [196]:
# load document
dir = 'data/Morgan-Stanley_2019-Sustainability-Report_Final.pdf'
df = pd.DataFrame({'Ticker':'Barclays', 'content': preprocess.extract_content(open_pdf_file=dir)}, index=[0])
paragraphs = preprocess.extract_statements(text=df['content'].values.tolist()[0], n_word_paragraph=200)

In [11]:
# load models
topic_pipeline = pipeline.topic_load_pipeline(model_dir="joeddav/xlm-roberta-large-xnli")
sentiment_pipeline = pipeline.sentiment_load_pipeline(model_dir="distilbert-base-uncased-finetuned-sst-2-english")

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [278]:
topics = ["ESG","Food", "Finance", "Social"]
nr_paragraphs = 3
predictions_all = pipeline.predict(topic_pipeline=topic_pipeline, sentiment_pipeline=sentiment_pipeline, doc=paragraphs[0:nr_paragraphs], topics=topics)
pipeline.format_topic_sent(dic=predictions_all, doc=paragraphs[0:nr_paragraphs])


Original text     Topic            \
                                                        Social       ESG   
0  Sustainability Report TABLE OF CONTENTS 3Our S...  0.929429  0.869356   
1  Committed to facilitate the prevention, reduct...  0.550831  0.449156   
2  Introduced a new Sustainability at Work progra...  0.488687  0.933936   

                      Sentiment            
    Finance      Food  NEGATIVE  POSITIVE  
0  0.397646  0.001796  0.003226  0.996774  
1  0.199675  0.001098  0.021890  0.978110  
2  0.001308  0.000271  0.001795  0.998205

In [295]:
# weighted by numbers of words (weighted average)
pred_topic = predictions_all["Topic"]
df = pd.DataFrame(paragraphs[0:nr_paragraphs], columns=["Original text"])
df["n_words"] = df["Original text"].apply(lambda x: len(x.split()))
(pd.DataFrame(pred_topic.multiply(df["n_words"], axis="index").sum()/df["n_words"].sum(),columns=["Confidence %"])*100).sort_values(by="Confidence %",ascending=False).round(3)

,Confidence %
Social,80.695
ESG,79.171
Finance,31.614
Food,0.150


In [193]:
# simple average
(pd.DataFrame(predictions_all["Topic"].mean(),columns=['Confidence (%)'])*100).round(3).sort_values(by='Confidence (%)',ascending=False)

,Confidence (%)
ESG,75.082
Social,65.632
Finance,19.954
Food,0.106


In [296]:
# weighted average sentiment
pred_sent = predictions_all["Sentiment"]
(pd.DataFrame(pred_sent.multiply(df["n_words"], axis="index").sum()/df["n_words"].sum(),columns=["Confidence %"])*100).sort_values(by="Confidence %",ascending=False).round(3)

,Confidence %
POSITIVE,99.317
NEGATIVE,0.683


In [301]:
# simple average
(pd.DataFrame(predictions_all["Sentiment"].mean(), columns=["Confidence %"])*100).sort_values(by="Confidence %",ascending=False).round(3)

,Confidence %
POSITIVE,99.103
NEGATIVE,0.897


In [326]:
# Show weighted normalized sentiment predictions 
pipeline.compute_weighted_sentiment(dic=predictions_all, nr_words=df["n_words"], weighted_by_words=True)

Confidence (%)
NEGATIVE Social            0.569
         ESG               0.518
         Finance           0.559
         Food              0.595
POSITIVE Social           99.431
         ESG              99.482
         Finance          99.441
         Food             99.405